In [ ]:
!pip install git+git://github.com/smart-on-fhir/client-py.git

Erstellen einer Observation-Ressource zur Angabe des Blutdrucks

In [1]:
from IPython.display import IFrame
import json
from json import dumps
import fhirclient
import fhirclient.models.observation as obx
import fhirclient.models.meta as ma
import fhirclient.models.codeableconcept as cc
import fhirclient.models.coding as co
import fhirclient.models.fhirreference as fr
import fhirclient.models.fhirdate as fd
import fhirclient.models.quantity as q

In [2]:
# Blutdruck Profil in der FHIR Spezifikation.
IFrame('https://hl7.org/fhir/bp.html', width=1200, height=330)

In [3]:
# Instanziieren der Observation
observation = obx.Observation()

In [4]:
# Hinzufügen von Metadaten über die Ressource
meta = ma.Meta()
meta.profile = ['http://hl7.org/fhir/StructureDefinition/bp']
observation.meta = meta

In [5]:
# Status der Blutdruckmessung
status = 'final'
observation.status = status

In [6]:
# Kategorie der Observation
category = cc.CodeableConcept()
categoryCoding = co.Coding()
categoryCoding.system = 'http://terminology.hl7.org/CodeSystem/observation-category'
categoryCoding.code = 'vital-signs'
category.coding = [categoryCoding]
observation.category = [category]

In [7]:
# Der LOINC Code für diese Observation
code = cc.CodeableConcept()
codeCoding = co.Coding()
codeCoding.system = 'http://loinc.org'
codeCoding.code = '85354-9'
codeCoding.display = 'Blood pressure panel with all children optional'
code.coding = [codeCoding]
observation.code = code

In [8]:
# Referenz zum Patienten
subject = fr.FHIRReference()
subject.reference = 'Patient/example'
observation.subject = subject

In [9]:
# Zeitpunkt der Messung
date = fd.FHIRDate('2020-12-02')
observation.effectiveDateTime = date

In [10]:
print(json.dumps(observation.as_json(), indent=4))

{
    "meta": {
        "profile": [
            "http://hl7.org/fhir/StructureDefinition/bp"
        ]
    },
    "category": [
        {
            "coding": [
                {
                    "code": "vital-signs",
                    "system": "http://terminology.hl7.org/CodeSystem/observation-category"
                }
            ]
        }
    ],
    "code": {
        "coding": [
            {
                "code": "85354-9",
                "display": "Blood pressure panel with all children optional",
                "system": "http://loinc.org"
            }
        ]
    },
    "effectiveDateTime": "2020-12-02",
    "status": "final",
    "subject": {
        "reference": "Patient/example"
    },
    "resourceType": "Observation"
}


Aufgabe

In [11]:
# Noch fehlen bei dieser Blutdruckmessung die Messwerte.
# Überprüfen Sie die Vorgaben im FHIR Blutdruck-Profil: https://hl7.org/fhir/bp.html
# Implementieren Sie die fehlenden Komponenten, sodass systolischer- und diastolischer-Wert vorhanden sind. 
# Tipp:
# component = obx.ObservationComponent()

In [13]:
# systolischer Blutdruck
systolicComponent = obx.ObservationComponent()

systolicCode = cc.CodeableConcept()
systolicCodeCoding = co.Coding()
systolicCodeCoding.system = 'http://loinc.org'
systolicCodeCoding.code = '8480-6'
systolicCodeCoding.display = 'Systolic blood pressure'
systolicCode.coding = [systolicCodeCoding]

systolicComponent.code = systolicCode

systolicValue = q.Quantity()
systolicValue.value = 120
systolicValue.unit = 'mmHg'
systolicValue.system = 'http://unitsofmeasure.org'
systolicValue.code = 'mm[Hg]'

systolicComponent.valueQuantity = systolicValue

# diastolischer Blutdruck
diastolicComponent = obx.ObservationComponent()

diastolicCode = cc.CodeableConcept()
diastolicCodeCoding = co.Coding()
diastolicCodeCoding.system = 'http://loinc.org'
diastolicCodeCoding.code = '8462-4'
diastolicCodeCoding.display = 'Diastolic blood pressure'
diastolicCode.coding = [diastolicCodeCoding]

diastolicComponent.code = diastolicCode

diastolicValue = q.Quantity()
diastolicValue.value = 80
diastolicValue.unit = 'mmHg'
diastolicValue.system = 'http://unitsofmeasure.org'
diastolicValue.code = 'mm[Hg]'

diastolicComponent.valueQuantity = diastolicValue

observation.component = [systolicComponent, diastolicComponent]

In [14]:
print(json.dumps(observation.as_json(), indent=4))

{
    "meta": {
        "profile": [
            "http://hl7.org/fhir/StructureDefinition/bp"
        ]
    },
    "category": [
        {
            "coding": [
                {
                    "code": "vital-signs",
                    "system": "http://terminology.hl7.org/CodeSystem/observation-category"
                }
            ]
        }
    ],
    "code": {
        "coding": [
            {
                "code": "85354-9",
                "display": "Blood pressure panel with all children optional",
                "system": "http://loinc.org"
            }
        ]
    },
    "component": [
        {
            "code": {
                "coding": [
                    {
                        "code": "8480-6",
                        "display": "Systolic blood pressure",
                        "system": "http://loinc.org"
                    }
                ]
            },
            "valueQuantity": {
                "code": "mm[Hg]",
                "syst

In [15]:
# Validieren gegen den FHIR Server
import requests
from requests import get, post, put

In [16]:
fhir_test_server = 'https://vonk.fire.ly'

headers = {
'Accept':'application/fhir+json; fhirVersion=4.0',
'Content-Type':'application/fhir+json; fhirVersion=4.0'
}

In [17]:
response = requests.post(f'{fhir_test_server}/Observation/$validate', headers = headers, data = dumps(observation.as_json()))
data = response.json()

In [18]:
print(response.status_code)

200


In [19]:
print(json.dumps(data, indent=4))

{
    "resourceType": "OperationOutcome",
    "id": "0b80490f-3628-415a-91e6-8e8b37cb34b9",
    "meta": {
        "versionId": "2055bf62-e033-4893-916f-a77a4290721b",
        "lastUpdated": "2020-11-24T21:49:14.878+00:00"
    },
    "issue": [
        {
            "severity": "information",
            "code": "informational",
            "diagnostics": "The operation was successful"
        }
    ]
}
